In [ ]:
import json
import numpy as np
import yfinance as yf
import random
random.seed(0)
np.random.seed(0)
from pytickersymbols import PyTickerSymbols

def is_valid_ticker(ticker):
    """Check if a ticker exists by retrieving 1 day of historical data."""
    try:
        data = yf.Ticker(ticker).history(period="1d")
        return not data.empty  # Valid if data is not empty
    except:
        return False  # Invalid ticker
    

def randomly_assign_qubits(n_qubits, stocks):
    """
    Randomly assign qubits to stocks, ensuring each stock gets at least one qubit.
    
    Args:
        n_qubits: Total number of qubits
        stocks: List of stock tickers
        
    Returns:
        qubits_per_stock: Dictionary with number of qubits per stock
    """
    n_stocks = len(stocks)
    if n_qubits < n_stocks and n_qubits != n_stocks:
        raise ValueError("Number of qubits must be at least equal to number of stocks")
    
    # Start by giving each stock one qubit
    qubits_per_stock = {stock: 1 for stock in stocks}
    
    # Randomly distribute remaining qubits
    remaining_qubits = n_qubits - n_stocks
    
    # Use random.choices with weights to distribute remaining qubits
    for _ in range(remaining_qubits):
        # Randomly select a stock to give an additional qubit
        chosen_stock = np.random.choice(stocks)
        qubits_per_stock[chosen_stock] += 1
    
    return qubits_per_stock

stock_data = PyTickerSymbols()
stocks = list(stock_data.get_dow_jones_nyc_yahoo_tickers())
stocks = [s for s in stocks if is_valid_ticker(s)]
print("Available stocks: ", stocks)

experiment_data = []

for n_stocks in range(2, 11):
    for max_qubits in range(n_stocks, 11):
        failed_attempts = 0
        successful_attempts = 0
        while True:

            sampled_stocks = random.sample(stocks, n_stocks)

            start = "2015-01-01"
            end = "2025-01-01"
            data = yf.download(sampled_stocks, start=start, end=end, progress=False)
            prices_now = data["Close"].iloc[-1]

            qubits = list(range(max_qubits))

            stock_metadata = {}
            
            # Divide qubits randomly for each stock so that they sum up to max_qubits
            qubits_per_stock = randomly_assign_qubits(max_qubits, sampled_stocks)
            budget = 0
            for stock in sampled_stocks:
                ints_for_stock = list(range(int("1"*qubits_per_stock[stock], 2) + 1))
                random_int = np.random.choice(ints_for_stock)
                budget += prices_now[stock]*random_int

                stock_metadata[stock] = {
                    "max_int": ints_for_stock[-1],
                    "qubits": qubits_per_stock[stock],
                    "random_int": int(random_int),
                    "price": float(prices_now[stock])
                }
            print("Budget: ", budget)

            if budget < 10:
                print("Skipping experiment because budget is too low")
                failed_attempts += 1
                continue
            
            # Verify that the number of qubits is low enough
            total_qubits = 0
            for asset in sampled_stocks:
                N = int(np.ceil(np.log2(np.ceil(budget/prices_now[asset]))))
                total_qubits += N
            
            if total_qubits > 12:
                print("Skipping experiment because total qubits is too high")
                failed_attempts += 1
                continue
            
            data_point = {}
            data_point["max_qubits"] = max_qubits
            data_point["budget"] = budget
            data_point["stock_metadata"] = stock_metadata
            data_point["stocks"] = sampled_stocks
            data_point["start"] = start
            data_point["end"] = end
            data_point["n_stocks"] = n_stocks
        
            experiment_data.append(data_point)
            successful_attempts += 1

            if successful_attempts == 10:
                break
            elif failed_attempts == 100:
                print("Failed 100 attempts, skipping to next configuration")
                break

with open("experiments_data.json", "w") as f:
    json.dump({"data": experiment_data}, f, indent=4)

$GS-PK: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Available stocks:  ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'NKE', 'MRK', 'MSFT', 'PG', 'TRV', 'UNH', 'VZ', 'V', 'WMT', 'WBA', 'DIS', 'AMGN', 'HON', 'CRM', 'DOW']


ValueError: This script will overwrite the experiments_data.json file. If you want to proceed, remove this line.